# WP1 - Anomaly detection

In [1]:
import py2neo
import networkx as nx
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import grangercausalitytests

In [2]:
graph = py2neo.Graph("bolt://phoenix:7687", auth=("neo4j", ""))

In [3]:
graph_rel = graph.relationships.match().all()

### Calculate granger causality on dataset and update graph

In [16]:
MAXLAG = 2
f = '../data/loadtest-Updated.csv'
P = pd.read_csv(f).sort_values(by=['Time', 'Unnamed: 0'])

#perform Granger-Causality test
for rel in graph_rel:
    source = rel.start_node['id']
    target = rel.end_node['id']
    if source in P.columns and target in P.columns:
        gc_dicts = grangercausalitytests(P[[source, target]], MAXLAG, verbose=False)
        p_values = [lag_result[0]['ssr_ftest'][1] for lag_result in gc_dicts.values()]
        print(source, target)
        if np.min(p_values) < 0.05:
            print('causal')
            rel["causal"] = True
            graph.push(rel)
        else:
            print('not causal')
            rel["causal"] = False
            graph.push(rel)
           

carts front-end
not causal
user user-db
causal
user front-end
not causal
catalogue front-end
not causal
catalogue-db catalogue
not causal
orders-db orders
causal
carts-db carts
causal
orders shipping
causal
orders front-end
not causal
shipping queue-master
causal


In [ ]:
neighbor_q = '''MATCH (a)-[r]-(b) where a.id='worker1' RETURN b'''
worker_neighbors = [i.get('b') for i in graph.run(neighbor_q).data()]

In [27]:
for n1 in worker_neighbors:
    for n2 in worker_neighbors:
        source = n1['id']
        target = n2['id']
        if source == target:
            continue
        if source in P.columns and target in P.columns:
            gc_dicts = grangercausalitytests(P[[source, target]], MAXLAG, verbose=False)
            p_values = [lag_result[0]['ssr_ftest'][1] for lag_result in gc_dicts.values()]
            print(source, target)
            if np.min(p_values) < 0.05:
                print('causal')
            else:
                print('not causal')

rabbitmq queue-master
causal
rabbitmq shipping
causal
rabbitmq catalogue-db
not causal
rabbitmq catalogue
causal
rabbitmq user
causal
rabbitmq carts
causal
rabbitmq payment
causal
rabbitmq orders
causal
queue-master rabbitmq
causal
queue-master shipping
causal
queue-master catalogue-db
not causal
queue-master catalogue
causal
queue-master user
not causal
queue-master carts
not causal
queue-master payment
not causal
queue-master orders
causal
shipping rabbitmq
causal
shipping queue-master
causal
shipping catalogue-db
causal
shipping catalogue
not causal
shipping user
not causal
shipping carts
causal
shipping payment
not causal
shipping orders
causal
catalogue-db rabbitmq
not causal
catalogue-db queue-master
causal
catalogue-db shipping
not causal
catalogue-db catalogue
not causal
catalogue-db user
causal
catalogue-db carts
causal
catalogue-db payment
not causal
catalogue-db orders
not causal
catalogue rabbitmq
not causal
catalogue queue-master
causal
catalogue shipping
not causal
catalo